In [1]:
import torch
import base_func 
import numpy as np
import torch.nn as nn
from tqdm import tqdm
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# 讀取數據
train_normal = np.load('Data/train_normal.npy')
train_outer_break = np.load('Data/train_outer_break.npy')
train_inner_break = np.load('Data/train_inner_break.npy')

# 創建標籤
labels_normal = np.zeros(len(train_normal))
labels_outer_break = np.ones(len(train_outer_break))
labels_inner_break = np.full(len(train_inner_break), 2)

# 合併數據和標籤
X_train = np.concatenate((train_normal, train_outer_break, train_inner_break), axis=0)
# X_train = resize_data(X_train, 200)
y_train = np.concatenate((labels_normal, labels_outer_break, labels_inner_break), axis=0)
print(X_train.shape)
print(y_train.shape)

X_train_tensor = torch.tensor(X_train, dtype=torch.float).unsqueeze(1)  # Adding channel dimension
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Assuming classification task

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
batch_size = 1  # Adjust the batch size according to your needs and hardware capabilities

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



(750, 10240)
(750,)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out


In [3]:

# model_list = {
#     "VibrationCNN" : lambda: VibrationCNN(),
# }
# model_name = "VibrationCNN"
# loaders = base_func.get_dataloaders(train_dataset, 0.6, 0.5, 1)
# phases = ['train','val']
# base_func.train(model_list, "VibrationCNN", loaders, phases)

In [4]:
loaders = base_func.get_dataloaders(train_dataset, 0.6, 0.5, 1)


--------- INDEX checking ---------
Original: [0, 1, 2, 3, 4]
Shuffled: [437, 304, 207, 119, 204]
--------- INDEX shuffled ---------

Total number of samples: 750 datapoints
Number of train samples: 450 batches/ 450 datapoints
Number of val samples: 150 batches/ 150 datapoints
Number of test samples: 150 batches/ 150 datapoints



In [5]:
# Hyperparameters
sequence_length = 1024
input_size = 1
hidden_size = 128  # Can be adjusted
num_layers = 2  # Can be adjusted
num_classes = 3
learning_rate = 0.001
num_epochs = 20
batch_size = 1  # Assuming your data loader uses this batch size

# Initialize the model, loss criterion, and optimizer
model = SimpleRNN(input_size, hidden_size, num_layers, num_classes).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Assuming you have a DataLoader for your training data
for epoch in range(num_epochs):
    for i, (sequences, labels) in tqdm(enumerate(loaders['train'])):  # Example DataLoader
        sequences = sequences.view(-1, sequence_length, input_size).cuda()
        labels = labels.cuda()

        # Forward pass
        outputs = model(sequences)
        # print(labels.shape)
        # print(outputs.shape)
        loss = 0
        for j in range(10):  # Iterate over the 4 targets
            # Compute loss for each target, assuming labels are long and contain indices
            # print(outputs[i, :].reshape(1,-1).shape)
            loss += criterion(outputs[j, :].reshape(1,-1), labels)
        # loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {loss.item():.4f}')


128it [00:00, 217.05it/s]

Epoch [1/20], Step [100], Loss: 12.5871


228it [00:01, 227.52it/s]

Epoch [1/20], Step [200], Loss: 11.8351


328it [00:01, 228.34it/s]

...

Epoch [5/20], Step [200], Loss: 20.0886


324it [00:01, 242.39it/s]

Epoch [5/20], Step [300], Loss: 5.2188


395it [00:01, 245.46it/s]


KeyboardInterrupt: 

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Initialize your model, loss criterion, and optimizer
model = SimpleRNN(input_size, hidden_size, num_layers, num_classes).cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.005
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for i, (sequences, labels) in enumerate(tqdm(loaders['train'])):  # Assuming loaders['train'] is your DataLoader
        sequences = sequences.view(-1, sequence_length, input_size).cuda()
        labels = labels.cuda()
        
        # Forward pass
        outputs = model(sequences)  # Assuming the output is of shape [batch_size, num_classes]

        # Calculate loss
        loss = 0
        for j in range(10):
            loss += criterion(outputs[j, :].reshape(1,-1), labels)
        total_loss += loss.item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs[-1, :].reshape(1,-1).data, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Print loss periodically
        # if (i + 1) % 100 == 0:
        #     print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}], Loss: {loss.item():.4f}')

    # Print accuracy for the epoch
    epoch_accuracy = 100 * correct_predictions / total_samples
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(loaders["train"]):.4f}, Accuracy: {epoch_accuracy:.2f}%')


100%|██████████| 450/450 [00:02<00:00, 174.56it/s]


Epoch [1/20], Loss: 14.0762, Accuracy: 36.00%


100%|██████████| 450/450 [00:02<00:00, 194.62it/s]


Epoch [2/20], Loss: 13.5697, Accuracy: 35.78%


100%|██████████| 450/450 [00:02<00:00, 195.94it/s]


...

Epoch [18/20], Loss: 13.5604, Accuracy: 35.78%


100%|██████████| 450/450 [00:02<00:00, 194.62it/s]


Epoch [19/20], Loss: 13.5603, Accuracy: 35.78%


100%|██████████| 450/450 [00:02<00:00, 199.99it/s]

Epoch [20/20], Loss: 13.5616, Accuracy: 36.00%
